# Check file shape e file mancanti

Questo paragrafo di codice permette verificare i file presenti nella cartella /Dataset/Dati.

Questa cartella contiene una cartella con nome nel formato "AAAA MM" per ogni mese di ogni anno.

Ogni cartella "/Dataset/Dati/AAAA MM" contiene un file csv per ogni giorno del mese con nome "Plant_F40_AAAA_MM_GG.csv".

In [ ]:
import pandas as pd
import os
from datetime import datetime
import calendar

In [ ]:
dataset_base_path = os.path.join("Dataset", "Dati")
final_dataset_path = "Dataset"

In [ ]:
def missing_days(y, m, list_of_days):
    tmp = calendar.monthrange(y, m)[1]
    total_days = set(range(1,tmp+1))
    list_of_days = set(list_of_days)
    return total_days.difference(list_of_days)

In [ ]:
check_file_shape = {}
check_dir = {}

for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    days = []
    for filename in os.listdir(dir_files):
    
        plant, f40, year, month, day = filename.split(".")[0].split("_")
        
        days.append(int(day))
        
        file_path = os.path.join(dir_files, filename)
        df_tmp = pd.read_csv(file_path, delimiter="\t")  
        
        if not df_tmp.shape[0] == 5760:
            check_file_shape[file_path] = df_tmp.shape

    miss_days = missing_days(int(year), int(month), days)
    if len(miss_days) > 0:
        check_dir[year+" "+month] = miss_days

In [ ]:
check_file_shape

In [ ]:
check_dir

# Riempimento giornate mancanti

Per ogni giorno della settima si sceglie il file più rappresentativo, poi manualmente si copiano questi file al posto dei giorni mancanti.

In [ ]:
import pandas as pd
import os
from datetime import datetime
import calendar
import numpy as np
import shutil

In [ ]:
dataset_base_path = os.path.join("Dataset", "Dati")
final_dataset_path = "Dataset"

In [ ]:
fields = [
    "P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM",
    "P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM",
    "P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM",
    "DayOfWeek"
]

In [ ]:
def make_timestamp(field, year, month, day):
    time, ampm = field.split(" ")
    h, m, s = time.split(":")
    if h == "12":
        h = "00"
    if ampm == "PM":
        h = str(int(h)+12)
    dtime = datetime(int(year), int(month), int(day), int(h), int(m), int(s))
    return str(dtime)

In [ ]:
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):
        
        #print(filename)
        
        # filename = Plant_F40_2021_06_01
        plant, f40, year, month, day = filename.split(".")[0].split("_")
        
        file_path = os.path.join(dir_files, filename)
        
        df_tmp = pd.read_csv(file_path, delimiter="\t")
        df_tmp["Time"] = df_tmp["Time"].apply(lambda x: make_timestamp(x, year, month, day))
        
        # "2011-02-02 11:00:00"
        date_format = "%Y-%m-%d %H:%M:%S"
        df_tmp["Timestamp"] = df_tmp["Time"].apply(lambda x: datetime.strptime(x, date_format).timestamp())
        df_tmp["Year"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[0])
        df_tmp["Month"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[1])
        df_tmp["Day"] = df_tmp["Time"].apply(lambda x: x.split(" ")[0].split("-")[2])
        df_tmp["DayOfWeek"] = df_tmp.apply(day_of_week, axis=1)
        
        df_tmp = df_tmp.loc[:, fields]        
        
        df = pd.concat([df, df_tmp])

In [ ]:
path = os.path.join(final_dataset_path, "dataset.csv")
df.to_csv(path, index=False)

In [ ]:
path = os.path.join(final_dataset_path, "dataset.csv")
df = pd.read_csv(path)

In [ ]:
fields = [
    "P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM",
    "P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM",
    "P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"
]

In [ ]:
df_mean = df.groupby(["DayOfWeek"])[fields].mean()

In [ ]:
def distance(mean_of_day, day_of_week):
    sum_of_squares = 0
    for field in fields:
        diff = df_mean.loc[day_of_week][field] - mean_of_day[field]
        sum_of_squares += diff ** 2
    return np.sqrt(sum_of_squares)

In [ ]:
best_file = [("",np.inf),("",np.inf),("",np.inf),("",np.inf),("",np.inf),("",np.inf),("",np.inf)]

for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):

        plant, f40, year, month, day = filename.split(".")[0].split("_")
        day_of_week = datetime(int(year), int(month), int(day)).weekday()

        file_path = os.path.join(dir_files, filename)
        df_tmp = pd.read_csv(file_path, delimiter="\t")
        df_tmp = df_tmp.loc[:, fields]
        
        mean_of_day = df_tmp.describe().loc["mean"]
        dist = distance(mean_of_day, day_of_week)
        
        if dist < best_file[day_of_week][1]:
            best_file[day_of_week] = (file_path, dist)

In [ ]:
best_file

In [ ]:
dataset_base_path = os.path.join("Dataset", "DatiFill")
for year_month, days in check_dir.items():
    for day in days:
        
        path = os.path.join(dataset_base_path, year_month)
        year, month = year_month.split()
        filename = "Plant_F40_" + year + "_" + month + "_" + str(day).zfill(2) + ".csv" 
        path = os.path.join(path, filename)
        
        day_of_week = datetime(int(year), int(month), int(day)).weekday()
        original = best_file[day_of_week][0]
        
        shutil.copyfile(original, path)       

# Costruzione dataset per analisi temporale (plot)

Questo paragrafo permette di costruire un dataset "/Dataset/dataset_dummy.csv" riempendo i "buchi", cioè inserendo dei valori NaN per tutte quelle giornate per cui non è presente il file.

Questo dataset permette di fare i plot delle curve di carico, considerando anche le giornate per cui non si dispongono dei dati.

Inoltre dal momento che il dataset presenta molti outlier è possibile impostare dei range di variazione per i vari carichi, in modo che se la potenza supera questo range l'outlier viene sostituito attraverso un valore ottenuto per interpolazione.

**Operazione preliminare**: cancellare la cartella /Dataset/Dummy e fare una copia di /Dataset/DatiFill rinominando la cartella in /Dataset/Dummy.

In [1]:
import pandas as pd
import os
from datetime import datetime
import calendar
import numpy as np

In [2]:
start_aaaa = 2020
end_aaaa = 2023

In [3]:
dataset_base_path = os.path.join("Dataset", "Dummy")
final_dataset_plot_path = "Dataset"

In [4]:
def make_timestamp(field, year, month, day):
    time, ampm = field.split(" ")
    h, m, s = time.split(":")
    if h == "12":
        h = "00"
    if ampm == "PM":
        h = str(int(h)+12)
    dtime = datetime(int(year), int(month), int(day), int(h), int(m), int(s))
    return str(dtime)

In [5]:
def day_of_week(x):
    date = datetime(int(x["Year"]), int(x["Month"]), int(x["Day"]))
    return date.weekday()

In [6]:
fields = [
    "Time", "Timestamp", "DayOfWeek","Year", "Month", "Day", "Hour", "Minute", "Second",
    "P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM",
    "P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM",
    "P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"
]

L1 = ["P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM"]
L2 = ["P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM"]
L3 = ["P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"]

In [7]:
min_power = {
    "P_CabinePower_L1": 50, "P L1 WS Emergency Light": 150, "P L1 WS Light": 200, "P L1 WS FanCoil": 0, "P L1 WS FM": 0,
    "P_HeatPump": 0, "P L2 WS Emergency Light": 200, "P L2 WS Light": 200, "P L2 WS FanCoil": 0, "P L2 WS FM": 0,
    "P_CabineLight_L3": 0, "P L3 WS Emergency Light": 200, "P L3 WS Light": 250, "P L3 WS FanCoil": 0, "P L3 WS FM": 200
}
max_power = {
    "P_CabinePower_L1": 150, "P L1 WS Emergency Light": 1500, "P L1 WS Light": 350, "P L1 WS FanCoil": 200, 
    "P L1 WS FM": 4000, "P_HeatPump": 1000, "P L2 WS Emergency Light": 1700, "P L2 WS Light": 400, 
    "P L2 WS FanCoil": 200, "P L2 WS FM": 1300, "P_CabineLight_L3": 10, "P L3 WS Emergency Light": 1000, 
    "P L3 WS Light": 350, "P L3 WS FanCoil": 200, "P L3 WS FM": 2000
}

In [8]:
for year in range(start_aaaa,end_aaaa):
    for month in range(1,13):  
        dir_files = os.path.join(dataset_base_path, str(year)+" "+str(month).zfill(2))
        if not os.path.exists(dir_files):
            os.mkdir(dir_files)
        for day in range(1,calendar.monthrange(year, month)[1]+1):
            # Plant_F40_2022_03_01.csv
            filename = "Plant_F40_" + str(year) + "_" + str(month).zfill(2) + "_" + str(day).zfill(2) + ".csv"
            path = os.path.join(dir_files, filename)
            data = []
            if not os.path.exists(path):
                for hour in range(0,24):
                    for minute in range(0,60):
                        for second in [11, 26, 41, 56]:
                            dtime = datetime(year, month, day, hour, minute, second)
                            data.append({
                                "Time": str(dtime), 
                                "Timestamp": dtime.timestamp(), 
                                "DayOfWeek": datetime(year, month, day).weekday(),
                                "Year": year, 
                                "Month": month, 
                                "Day": day, 
                                "Hour": hour, 
                                "Minute": minute, 
                                "Second": second,
                                "P_CabinePower_L1": np.nan, 
                                "P L1 WS Emergency Light": np.nan, 
                                "P L1 WS Light": np.nan, 
                                "P L1 WS FanCoil": np.nan, 
                                "P L1 WS FM": np.nan,
                                "P_HeatPump": np.nan, 
                                "P L2 WS Emergency Light": np.nan, 
                                "P L2 WS Light": np.nan, 
                                "P L2 WS FanCoil": np.nan, 
                                "P L2 WS FM": np.nan,
                                "P_CabineLight_L3": np.nan, 
                                "P L3 WS Emergency Light": np.nan, 
                                "P L3 WS Light": np.nan, 
                                "P L3 WS FanCoil": np.nan, 
                                "P L3 WS FM": np.nan
                            })
                df = pd.DataFrame(data)
                df.to_csv(path, index=False)
                
            else:
                df = pd.read_csv(path, delimiter="\t")
                df["Time"] = df["Time"].apply(lambda x: make_timestamp(x, year, month, day))

                # "2011-02-02 11:00:00"
                date_format = "%Y-%m-%d %H:%M:%S"
                df["Timestamp"] = df["Time"].apply(lambda x: datetime.strptime(x, date_format).timestamp())
                df["Year"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[0])
                df["Month"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[1])
                df["Day"] = df["Time"].apply(lambda x: x.split(" ")[0].split("-")[2])
                df["Hour"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[0])
                df["Minute"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[1])
                df["Second"] = df["Time"].apply(lambda x: x.split(" ")[1].split(":")[2])
                df["DayOfWeek"] = df.apply(day_of_week, axis=1)
                
                for field in L1+L2+L3:
                    df[field] = df[field].where((df[field] >= min_power[field]) & (df[field] <= max_power[field]), np.nan)
                    df[field] = df[field].interpolate()
                
                df = df.loc[:, fields]
                df.to_csv(path, index=False)

In [9]:
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):
        file_path = os.path.join(dir_files, filename)
        
        df_tmp = pd.read_csv(file_path)
        df_tmp["PowerL1"] = df_tmp[L1].sum(axis=1) 
        df_tmp["PowerL2"] = df_tmp[L2].sum(axis=1)
        df_tmp["PowerL3"] = df_tmp[L3].sum(axis=1)
        df_tmp["TotalPower"] = df_tmp[L1+L2+L3].sum(axis=1)
        
        df = pd.concat([df, df_tmp])

In [10]:
df[["P L1 WS Emergency Light", "P L1 WS Light"]] = df[["P L1 WS Light", "P L1 WS Emergency Light"]]
df[["P L2 WS Emergency Light", "P L2 WS Light"]] = df[["P L2 WS Light", "P L2 WS Emergency Light"]]
df[["P L3 WS Emergency Light", "P L3 WS Light"]] = df[["P L3 WS Light", "P L3 WS Emergency Light"]]

In [11]:
df["PowerL1"].replace(0, np.nan, inplace=True)
df["PowerL2"].replace(0, np.nan, inplace=True)
df["PowerL3"].replace(0, np.nan, inplace=True)
df["TotalPower"].replace(0, np.nan, inplace=True)

In [12]:
display(df)

,Time,Timestamp,DayOfWeek,Year,Month,Day,Hour,Minute,Second,P_CabinePower_L1,...,P L2 WS FM,P_CabineLight_L3,P L3 WS Emergency Light,P L3 WS Light,P L3 WS FanCoil,P L3 WS FM,PowerL1,PowerL2,PowerL3,TotalPower
0,2020-01-01 00:00:11,1.577833e+09,2,2020,1,1,0,0,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 00:00:26,1.577833e+09,2,2020,1,1,0,0,26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 00:00:41,1.577833e+09,2,2020,1,1,0,0,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 00:00:56,1.577833e+09,2,2020,1,1,0,0,56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 00:01:11,1.577833e+09,2,2020,1,1,0,1,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,2022-12-31 23:58:56,1.672528e+09,5,2022,12,31,23,58,56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5756,2022-12-31 23:59:11,1.672528e+09,5,2022,12,31,23,59,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5757,2022-12-31 23:59:26,1.672528e+09,5,2022,12,31,23,59,26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5758,2022-12-31 23:59:41,1.672528e+09,5,2022,12,31,23,59,41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
path = os.path.join(final_dataset_plot_path, "dataset_dummy.csv")
df.to_csv(path, index=False)

# Costruzione dataset mediato orario

Questo paragrafo permette di costruire un dataset /Dataset/dataset_orario.csv mediato sull'ora, in particolare si fa una media dei valori nell'intervallo $[hh_t:00 - hh_t:59]$ in $hh_t:00$.

In [1]:
import pandas as pd
import os
from datetime import datetime
import calendar

In [2]:
dataset_base_path = os.path.join("Dataset", "Dummy")
final_dataset_path = "Dataset"

In [16]:
def make_date(row):
    return datetime(int(row["Year"]), int(row["Month"]), int(row["Day"]), int(row["Hour"])).timestamp()

In [17]:
fields = [
    "P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM",
    "P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM",
    "P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"
]

L1 = ["P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM"]
L2 = ["P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM"]
L3 = ["P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"]

In [18]:
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):
        
        file_path = os.path.join(dir_files, filename)
        df_tmp = pd.read_csv(file_path)
        
        df_tmp = df_tmp.groupby(["Year", "Month", "Day", "Hour"])[fields].mean()
        df_tmp = df_tmp.reset_index()
        df_tmp["Timestamp"] = df_tmp.apply(make_date, axis=1)
        df_tmp["PowerL1"] = df_tmp[L1].sum(axis=1) 
        df_tmp["PowerL2"] = df_tmp[L2].sum(axis=1)
        df_tmp["PowerL3"] = df_tmp[L3].sum(axis=1)
        df_tmp["TotalPower"] = df_tmp[L1+L2+L3].sum(axis=1)
        
        df = pd.concat([df, df_tmp])      

In [19]:
df["PowerL1"].replace(0, np.nan, inplace=True)
df["PowerL2"].replace(0, np.nan, inplace=True)
df["PowerL3"].replace(0, np.nan, inplace=True)
df["TotalPower"].replace(0, np.nan, inplace=True)

In [20]:
display(df)

,Year,Month,Day,Hour,P_CabinePower_L1,P L1 WS Emergency Light,P L1 WS Light,P L1 WS FanCoil,P L1 WS FM,P_HeatPump,...,P_CabineLight_L3,P L3 WS Emergency Light,P L3 WS Light,P L3 WS FanCoil,P L3 WS FM,Timestamp,PowerL1,PowerL2,PowerL3,TotalPower
0,2020,5,1,0,80.543650,283.208333,303.791667,0.000,114.958333,6.275392,...,0.000000,NaN,NaN,0.000000,773.000000,1.588284e+09,782.501983,896.650392,773.000000,2452.152375
1,2020,5,1,1,80.307533,281.166667,301.708333,0.000,274.208333,6.260633,...,0.000000,NaN,NaN,0.000000,804.916667,1.588288e+09,937.390867,888.718967,804.916667,2631.026500
2,2020,5,1,2,80.498729,284.750000,305.625000,0.000,114.583333,6.360458,...,0.000000,NaN,NaN,0.000000,767.041667,1.588291e+09,785.457063,889.193792,767.041667,2441.692521
3,2020,5,1,3,80.380096,281.083333,302.416667,0.000,371.583333,6.322979,...,0.000000,NaN,NaN,0.000000,770.708333,1.588295e+09,1035.463429,896.947979,770.708333,2703.119742
4,2020,5,1,4,80.597587,288.166667,308.541667,0.000,114.083333,6.570863,...,0.000000,NaN,NaN,0.000000,770.958333,1.588298e+09,791.389254,897.279196,770.958333,2459.626783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2022,5,31,19,83.660942,436.125000,260.708333,2.625,975.583333,375.136483,...,5.469125,358.416667,NaN,90.000000,1321.062500,1.654016e+09,1758.702608,2371.511483,1774.948292,5905.162383
20,2022,5,31,20,84.192696,425.541667,264.083333,0.000,860.583333,357.499563,...,5.612742,355.000000,NaN,90.000000,1014.250000,1.654020e+09,1634.401029,1976.749563,1464.862742,5076.013333
21,2022,5,31,21,84.676946,428.958333,265.416667,0.000,963.458333,334.256617,...,5.668521,357.916667,NaN,89.916667,941.250000,1.654024e+09,1742.510279,1890.756617,1394.751854,5028.018750
22,2022,5,31,22,85.116646,430.500000,268.833333,0.000,847.041667,287.612071,...,5.567238,359.416667,NaN,90.000000,910.833333,1.654027e+09,1631.491646,1839.695404,1365.817237,4837.004287


In [21]:
path = os.path.join(final_dataset_path, "dataset_orario.csv")
df.to_csv(path, index=False)

# Costruzione dataset mediato giornaliero

Questo paragrafo permette di costruire un dataset /Dataset/dataset_giornaliero.csv mediato sull'intera giornata.

In [ ]:
import pandas as pd
import os
from datetime import datetime
import calendar
import numpy as np

In [ ]:
dataset_base_path = os.path.join("Dataset", "Dummy")
final_dataset_path = "Dataset"

In [ ]:
def make_date(row):
    return datetime(int(row["Year"]), int(row["Month"]), int(row["Day"])).timestamp()

In [ ]:
fields = [
    "P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM",
    "P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM",
    "P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"
]

L1 = ["P_CabinePower_L1", "P L1 WS Emergency Light", "P L1 WS Light", "P L1 WS FanCoil", "P L1 WS FM"]
L2 = ["P_HeatPump", "P L2 WS Emergency Light", "P L2 WS Light", "P L2 WS FanCoil", "P L2 WS FM"]
L3 = ["P_CabineLight_L3", "P L3 WS Emergency Light", "P L3 WS Light", "P L3 WS FanCoil", "P L3 WS FM"]

In [ ]:
df = pd.DataFrame()
for dir_name in os.listdir(dataset_base_path):
    
    dir_files = os.path.join(dataset_base_path, dir_name)
    for filename in os.listdir(dir_files):
        
        file_path = os.path.join(dir_files, filename)
        
        df_tmp = pd.read_csv(file_path)
        
        df_tmp = df_tmp.groupby(["Year", "Month", "Day"])[fields].mean()
        df_tmp = df_tmp.reset_index()
        df_tmp["Timestamp"] = df_tmp.apply(make_date, axis=1)
        df_tmp["PowerL1"] = df_tmp[L1].sum(axis=1) 
        df_tmp["PowerL2"] = df_tmp[L2].sum(axis=1)
        df_tmp["PowerL3"] = df_tmp[L3].sum(axis=1)
        df_tmp["TotalPower"] = df_tmp[L1+L2+L3].sum(axis=1) 
        
        df = pd.concat([df, df_tmp])   

In [ ]:
df["PowerL1"].replace(0, np.nan, inplace=True)
df["PowerL2"].replace(0, np.nan, inplace=True)
df["PowerL3"].replace(0, np.nan, inplace=True)
df["TotalPower"].replace(0, np.nan, inplace=True)

In [ ]:
display(df)

In [ ]:
path = os.path.join(final_dataset_path, "dataset_giornaliero.csv")
df.to_csv(path, index=False)

# Costruzione dataset per training modelli

In [25]:
import pandas as pd
import os
from datetime import datetime
import calendar
import numpy as np

In [9]:
dataset_base_path = os.path.join("Dataset", "Dummy")
dummy_dataset_base_path = os.path.join("Dataset", "dataset_dummy.csv")
final_dataset_path = "Dataset"

In [ ]:
df = pd.read_csv(dummy_dataset_base_path)

In [32]:
df.columns

Index(['Time', 'Timestamp', 'DayOfWeek', 'Year', 'Month', 'Day', 'Hour',
       'Minute', 'Second', 'P_CabinePower_L1', 'P L1 WS Emergency Light',
       'P L1 WS Light', 'P L1 WS FanCoil', 'P L1 WS FM', 'P_HeatPump',
       'P L2 WS Emergency Light', 'P L2 WS Light', 'P L2 WS FanCoil',
       'P L2 WS FM', 'P_CabineLight_L3', 'P L3 WS Emergency Light',
       'P L3 WS Light', 'P L3 WS FanCoil', 'P L3 WS FM', 'PowerL1', 'PowerL2',
       'PowerL3', 'TotalPower'],
      dtype='object')

In [19]:
fields_for_training = ["PowerL1", "PowerL2", "PowerL3", "DayOfWeek"] # Holiday, Temperature, Month, Hour

In [34]:
df_tmp = df.groupby(["Year", "Month", "Day", "Hour"])[fields_for_training].mean()
df_tmp = df_tmp.dropna()
df_tmp = df_tmp.reset_index()
df_tmp["DayOfWeek"] = df_tmp["DayOfWeek"].astype(int)

In [35]:
df_tmp

,Year,Month,Day,Hour,PowerL1,PowerL2,PowerL3,DayOfWeek
0,2020,5,1,0,782.501983,896.650392,773.000000,4
1,2020,5,1,1,937.390867,888.718967,804.916667,4
2,2020,5,1,2,785.457063,889.193792,767.041667,4
3,2020,5,1,3,1035.463429,896.947979,770.708333,4
4,2020,5,1,4,791.389254,897.279196,770.958333,4
...,...,...,...,...,...,...,...,...
18259,2022,5,31,19,1758.702608,2371.511483,1774.948292,1
18260,2022,5,31,20,1634.401029,1976.749563,1464.862742,1
18261,2022,5,31,21,1742.510279,1890.756617,1394.751854,1
18262,2022,5,31,22,1631.491646,1839.695404,1365.817237,1


In [36]:
path = os.path.join(final_dataset_path, "dataset_addestramento.csv")
df_tmp.to_csv(path, index=False)